## What are Hyperparameters?

👉 These are **the settings you choose before training** a model.
They **control** how the model learns — not learned automatically.

Examples:

* For `DecisionTreeClassifier`:

  * `max_depth`, `min_samples_split`, `criterion`
* For `KNeighborsClassifier`:

  * `n_neighbors`, `weights`, `metric`
* For `SVM`:

  * `C`, `kernel`, `gamma`

Think of hyperparameters like the **"knobs"** you tune on a machine before running it.

---
## **1. GridSearchCV**
### Why Grid Search?

Normally, we’d guess:

```python
model = DecisionTreeClassifier(max_depth=3)
```

But what if `max_depth=5` or `min_samples_split=4` gives better accuracy?

That’s where **GridSearchCV** comes in. It tries **every possible combination** of parameters you give, trains the model on each, and picks the best one based on performance (using cross-validation internally).

---

## Real-life Analogy

Think of tuning your model like finding the best gym routine:
You test combinations like:

* Split type (Push/Pull/Legs)
* Reps (8, 10, 12)
* Rest time (30s, 60s, 90s)

You run all combos for a week → track results → pick the one that gives best gains.
That’s literally **Grid Search**.

---

## Example — Decision Tree Optimization


In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [9]:
X, y = load_iris(return_X_y=True)

In [10]:
model = DecisionTreeClassifier(random_state=42)

In [11]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 3, 4, 5, 6],
    'min_samples_split': [2, 3, 4],
}

# Grid Search with 5-fold CV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Best Parameters: {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}
Best Accuracy: 0.9733333333333334


**Output example:**

```
Best Parameters: {'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 2}
Best Accuracy: 0.9666
```

---

## 🧠 Behind the scenes

GridSearchCV does this under the hood:

* For each combo of parameters → Train using **5-fold cross-validation**
* Store the average accuracy
* Pick the highest-performing combo

So if your grid has 5 × 3 × 2 = 30 combos →
That’s 30 × 5 = **150 total model trainings** ⚙️

That’s why GridSearch is powerful but **computationally expensive**.

---

## 🧪 Step 4: How to use the best model

```python
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X)
```

---

## 🧰 Quick Tips

| Tip                       | Description                                     |
| ------------------------- | ----------------------------------------------- |
| ✅ Use small grid first    | Don’t test 1000 combos on first run             |
| ⚡ Use `n_jobs=-1`         | To use all CPU cores for faster search          |
| 💡 Combine with Pipelines | You can optimize preprocessing + model together |

---

## ⚡ TL;DR

> 🔹 Grid Search = brute-force tuning
> 🔹 Tries all combinations
> 🔹 Finds the most optimal hyperparameters
> 🔹 Expensive but accurate

---




## **2. RandomizedSearchCV**

### Grid Search vs Randomized Search

| Feature              | GridSearchCV                          | RandomizedSearchCV                       |
| -------------------- | ------------------------------------- | ---------------------------------------- |
| **Approach**         | Tests **every** parameter combination | Tests **random samples** of combinations |
| **Speed**            | Very slow (brute-force)               | Much faster (samples randomly)           |
| **Best for**         | Small grids / fewer parameters        | Large grids / expensive models           |
| **Accuracy**         | Precise but computationally heavy     | Approximate but efficient                |
| **Example use case** | Decision Tree, Logistic Regression    | Random Forest, XGBoost, SVM              |

So when your model has **many tunable parameters**, Grid Search becomes overkill.
That’s where RandomizedSearchCV shines 🚀

---

## Real-life Analogy

Grid Search → trying **every** protein shake recipe in the world 🍌🍫🥭🥤
Randomized Search → trying **20 random ones** and picking the best — much faster and you still find something awesome.

---

## Example: Random Forest Hyperparameter Tuning

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from scipy.stats import randint

In [13]:
X, y = load_iris(return_X_y=True)

In [14]:
model_2 = RandomForestClassifier(random_state=42)

In [15]:
param_dist = {
    "n_estimators": randint(50, 200),
    "max_depth": randint(2, 10),
    "min_samples_split": randint(2, 10),
    "min_samples_leaf": randint(1, 5),
    "criterion": ["gini", "entropy"],
}

# Randomized Search
rand_search = RandomizedSearchCV(
    model_2,
    param_distributions=param_dist,
    n_iter=20,  # number of random combos to try
    cv=5,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1,
)

rand_search.fit(X, y)

print("Best Params:", rand_search.best_params_)
print("Best Accuracy:", rand_search.best_score_)

Best Params: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 156}
Best Accuracy: 0.9666666666666668


**Output:**

```
Best Params: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 156}
Best Accuracy: 0.9666666666666668
```

---

## 🧠 Key Parameters

| Parameter             | Meaning                                  |
| --------------------- | ---------------------------------------- |
| `param_distributions` | dict or scipy.stats distributions        |
| `n_iter`              | how many random combos to test           |
| `cv`                  | folds for cross-validation               |
| `scoring`             | metric (e.g., accuracy, f1)              |
| `n_jobs=-1`           | use all CPU cores for faster computation |

---

## 🧠 When to Use Which

* Use **GridSearchCV** when you have few hyperparameters and want **precise tuning**.
* Use **RandomizedSearchCV** when you have **many parameters** and need **speed + efficiency**.

---

## ⚡ TL;DR

> 🔹 Both GridSearchCV & RandomizedSearchCV = hyperparameter tuning tools.

> 🔹 GridSearch = exhaustive, precise, but slow.

> 🔹 RandomizedSearch = faster, efficient, and surprisingly accurate.

---